In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz --user
nlp = en_core_web_sm.load()
import pandas as pd
import json
!pip install bs4 --user
from bs4 import BeautifulSoup
import requests
import re
import os
import mmap
import random
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy.util import minibatch, compounding

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [2]:
with open('data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[2]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0-I need to book a train that leaves after 08:45 on Sunday
1-I can help!  Where are you traveling to?
2-I'll be traveling to Cambridge and want to depart from Birmingham New Street. What are my options?
3-The TR9714 leaves at 09:40. 
4-Can you please provide me with the arrival and travel time? Thank you
5-The arrival time is 12:23, and it's a long trip - 163 minutes. Can I help you with anything else today?
6-Yes I am looking for colleges to visit in town.
7-Okay, there are 18 in town! What part of town would you like to go to?
8-Whichever one wherever you like best please. 
9-I personally like to visit Christ's College myself.  And, it's free to enter.  Would that work for you?
10-That sounds great.  Can you give me the address and phone number?
11-Their address is saint andrew's street, and their phone number 01223334900.
12-Great, thanks so much, that's all that I need!
13-Thank you so much for calling Cambridge TownInfo centre! It was a pleasure to serve you, we hope you enjoy you

## Load ontology

In [3]:
with open('ontology.json', 'r') as o:
    ontology = json.load(o)
    my_ent=list(ontology.keys())
    
    
action=['inform', 'request', 'select', 'recommend', 'not found', 'request booking', 'offer booking', 
        'inform booked', 'decline booking', 'welcome','greet', 'bye', 'reqmore' ]
topic= ['hotel', 'restaurant', 'taxi', 'train', 'attraction', 'police', 'hospital']

## display topics and actions

In [4]:
top=[]
act=[]
ont=list(ontology.keys())
for i in range(len(dict_json_weighted[object_1]['log'])):
    #print(dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        if str(x).lower() in topic:
            if str(x).lower() not in top:
                top.append(str(x).lower()) 
        if x.lemma_ in action:
            if x.lemma_ not in act:
                act.append(x.lemma_)
        #for z in ont:
            #if str(x) in str(ontology[z]):
                #print(z+" :"+str(x))           
print("topics: "+str(top))
print("actions: "+str(act))


topics: ['train']
actions: []


## Display the updated slots 

In [5]:
for i in dict_json_weighted[object_1]['goal']:
    if i not in['message', 'topic']:
        if dict_json_weighted[object_1]['goal'][i] !={}:
            print("****** "+i+" ******")
            for j in (dict_json_weighted[object_1]['goal'][i]):
                if dict_json_weighted[object_1]['goal'][i][j] not in ['fail_info', {}]:
                    print("°"+j)
                    print(dict_json_weighted[object_1]['goal'][i][j])
    

****** attraction ******
°reqt
['address', 'phone']
°info
{'type': 'college'}
****** train ******
°reqt
['trainID', 'arriveBy', 'duration']
°info
{'destination': 'cambridge', 'leaveAt': '08:45', 'departure': 'birmingham new street', 'day': 'sunday'}


# Custom NER

### create training sample

In [6]:
Train_data = [
     (
        "Hello, I am looking for a cheap place to dine in the centre", { "entities" :
        [(26, 19, "restaurant-price-range"), (46, 5, "restaurant-area")]
        }
    ),(
        "not really but i need it for 3 people",  { "entities" : 
        [(29, 8, "restaurant-book-people")]
        }
    ),(
        "I recommend La Raza - fantastic (and cheap!) Spanish cuisine. I'd be happy to book a table for you, if you like.",  { "entities" : 
        [(12, 7, "restaurant-name"), (46, 15, "restaurant-food")]
        }

    ),(
        "Yes for 3 at 14:45 wednesday, can I get the reference number too please", 
        { "entities" :
        [(8, 1, "restaurant-book-people"), (13, 5, "restaurant-book-time"), (19, 9, "restaurant-book-day")]
        }
    ),( 
        "Both offer free parking and free wifi. The Alexander B&B is 4 star rated. The El Shaddai has a 0 rating.", { "entities" : 
             [(11, 12, "hotel-parking"), (28, 9, "hotel-internet"), (39, 17, "hotel-name"), (60, 6, "hotel-stars"), (78, 11, "hotel-name"), (96, 8, "hotel-stars")]
             }
    ),( 
        "3 nights, 3 people, please.", { "entities" : 
             [(0, 8, "hotel-book-stay"), (10, 8, "hotel-book-people")]
             }
    ),( 
        "I would like a taxi from the hotel to the restaurant. I would like to arrive by 14:45. I would like the contact number and car type",  { "entities" : 
             [(77, 5, "taxi-arrive-by")]
             }
    ),( 
        "Thanks. I also need a cheap place to stay in the center.", { "entities" : 
             [(22, 19, "hotel-price-range"),(43, 13, "hotel-area")]
             }
    ),( 
        "I am looking for info about a hotel called city centre north b and b. Can you help?", { "entities" : 
             [(40, 25, "hotel-name")]
             }
    ),( 
        "Yes. City Centre North B and B is a guesthouse in the north side of town, that has both free internet and parking. Do you want to book a room?", { "entities" : 
             [(33, 10, "hotel-type"),(44, 25, "hoyel-area")]
             }
    ),( 
        "Ok, I am also looking for a train from Cambridge to Bishops Stortford.", { "entities" : 
             [(33, 14, "train-departure"),(47, 20, "train-destination")]
             }
    ),( 
        "I'll be leaving Monday after 15:30.", { "entities" : 
             [(16, 5, "train-day"),(23, 11, "train-leave-at")]
             }
    ),( 
        "TR2630 leaves at 17:29. Would you like for me to book the train for you?", { "entities" : 
             [(14, 8, "train-leave-at")]
             }
    ),( 
        "Yes I would like to book the train for 4 people and I'll need the reference number please.", { "entities" : 
             [(23, 8, "train-book-people")]
             }
    ),( 
        "I'm looking for a Mediterranean restaurant near the centre of town.", { "entities" : 
             [(18, 24, "restaurant-food"), (41, 23, "restaurant-area")]
             }
    ),( 
        "I've got three that fit that bill... the gardenia, la mimosa, and shiraz.  Do you have a preference?", { "entities" : 
             [(26, 16, "restaurant-name"),(39, 11, "restaurant-name"),(54, 5, "restaurant-name")]
             }
    ),( 
        "Great, could I book a table of 5 for 11:30 on wednesday? If so, please provide me with a reference number", { "entities" : 
             [(13, 12, "resraurant-book-people"),(29, 5, "restaurant-book-time"),(38, 9, "restaurant-bookday")]
             }
    ),( 
        "Is one of them in the cheap price range? ", { "entities" : 
             [(22, 5, "restaurant-price-range")]
             }
    ),( 
        "Yes. Can you please search for an attraction in the type of college?", { "entities" : 
             [(45, 15, "attraction-type")]
             }
    ),( 
        "Is an entrance fee okay? If so, the corpus christi college located at king's parade. Fee is 2 pounds. Would you like the phone number? ", { "entities" : 
             [(32, 26, "attraction-name")]
             }
    ),( 
        "I would like to leave Corpus Christi College in enough time to arrive at the restaurant at the booked time of 11:30. ", { "entities" : 
             [(22, 21, "train-departure"), (81, 20, "train-arrive-by")]
             }
    ),( 
        "I'm looking for a place to stay in Cambridge. It should include free wifi and be a 3 star.  Can you help me please? ", { "entities" : 
             [(62, 9, "hotel-internet"), (80, 6, "hotel-stars")]
             }
    ),( 
        "There are five to choose from. Would you like moderate or expensive price point? ", { "entities" : 
             [(45, 8, "hotel-price-range"), (57, 9, "hotel-price-range")]
             }
    ),( 
        "I have the gonville hotel. It is expensive. Would you like me to book that for you?", { "entities" : 
             [(7, 12, "hotel-name"), (31, 9, "hotel-price-range")]
             }
    ),( 
        "Do they have free parking? If so, please reserve a room for Tuesday for 7 people, 4 nights.", { "entities" : 
             [(13, 12, "hotel-parking"), (55, 7, "hotel-book-day"), (67, 8, "hotel-book-people"), (77,8, "hotel-book-stay")]
             }
    ),
]



### Training

In [13]:
output_dir2 = ("output_dir2")
def main_train(model=None, output_dir=r'model', n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in Train_data:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    #for i in my_ent:
     #   ner.add_label(i)

    # get names of other pipes to disable them during training
    #other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    #with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
    if model is None:
        nlp.begin_training()
    else:
        nlp.resume_training()
    #if model is None:
     #   optimizer = nlp.begin_training()
    #else:
     #   optimizer = nlp.entity.create_optimizer()
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER

        for itn in range(n_iter):
            print("Starting iteration" + str(itn))
            random.shuffle(Train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(Train_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.3,  # dropout - make it harder to memorise data
                losses=losses,
                )
                
            print("Losses", losses)
    new_model=nlp
    # test the trained model
    for text, _ in Train_data:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    #if output_dir2 is not None:
     #   output_dir2 = Path(output_dir2)
      #  if not output_dir2.exists():
       #     output_dir2.mkdir()
    nlp.to_disk(output_dir2)
    print("Saved model to", output_dir2)
main_train(model=None, output_dir=r'model', n_iter=100)

Created blank 'en' model
Starting iteration0
Losses {'ner': 465.93111550807953}
Starting iteration1
Losses {'ner': 159.95644586162763}
Starting iteration2
Losses {'ner': 1.9939078390485754}
Starting iteration3
Losses {'ner': 1.999998808135351}
Starting iteration4
Losses {'ner': 1.9999989271165997}
Starting iteration5
Losses {'ner': 1.9998061656968886}
Starting iteration6
Losses {'ner': 1.5467722900900993}
Starting iteration7
Losses {'ner': 11.472626180973483}
Starting iteration8
Losses {'ner': 3.027851985903351}
Starting iteration9
Losses {'ner': 1.3883270726457047}
Starting iteration10
Losses {'ner': 1.4310926443135628}
Starting iteration11
Losses {'ner': 0.93889004149102}
Starting iteration12
Losses {'ner': 1.9130931295428502}
Starting iteration13
Losses {'ner': 0.6867917114436607}
Starting iteration14
Losses {'ner': 11.848349512909158}
Starting iteration15
Losses {'ner': 0.7756770687846517}
Starting iteration16
Losses {'ner': 2.307178115092704}
Starting iteration17
Losses {'ner': 1.

### Testing the model

In [ ]:
def test_model(nlp):
    texts = [
        "Yes I would like to book the train for 4 people and I'll need the reference number please.",
        "I've got three that fit that bill... the gardenia, la mimosa, and shiraz.  Do you have a preference?",
        "Great, could I book a table of 5 for 11:30 on wednesday?",
    ]
    docs = nlp.pipe(texts)
    for doc in docs:
        print(doc.text)
        print([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != "-"])

In [14]:
test_model(nlp)

Yes I would like to book the train for 4 people and I'll need the reference number please.
[('Yes', 'intj', 'like'), ('I', 'nsubj', 'like'), ('would', 'aux', 'like'), ('like', 'ROOT', 'like'), ('to', 'aux', 'book'), ('book', 'xcomp', 'like'), ('the', 'det', 'train'), ('train', 'dobj', 'book'), ('for', 'prep', 'book'), ('4', 'nummod', 'people'), ('people', 'pobj', 'for'), ('and', 'cc', 'like'), ('I', 'nsubj', 'need'), ("'ll", 'aux', 'need'), ('need', 'conj', 'like'), ('the', 'det', 'number'), ('reference', 'compound', 'number'), ('number', 'dobj', 'need'), ('please', 'intj', 'need'), ('.', 'punct', 'need')]
I've got three that fit that bill... the gardenia, la mimosa, and shiraz.  Do you have a preference?
[('I', 'nsubj', 'got'), ("'ve", 'aux', 'got'), ('got', 'ROOT', 'got'), ('three', 'dobj', 'got'), ('that', 'nsubj', 'fit'), ('fit', 'relcl', 'three'), ('that', 'det', 'bill'), ('bill', 'dobj', 'fit'), ('...', 'punct', 'got'), ('the', 'det', 'gardenia'), ('gardenia', 'ROOT', 'gardenia')

In [10]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("3 nights, 3 people, please.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities [('3 nights', 'hotel-book-stay')]


In [18]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("Do they have free parking? If so, please reserve a room for Tuesday for 7 people, 4 nights.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities []


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [25]:
print("Loading from", output_dir2)
nlp2 = spacy.load(output_dir2)
doc = nlp2("I want to reserve 4 nights.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc,  style='ent')

Loading from output_dir2
Entities []


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


### Model evaluation

In [78]:
def evaluate(model, examples):
    scorer = Scorer()
    for text, metadata in Train_data:
        #print(text)
        doc_gold_text = model.make_doc(text)
        gold = GoldParse(doc_gold_text, entities=metadata['entities'])
        pred_value = model(text)
        scorer.score(pred_value, gold)
    return scorer.scores
test_data=[]
x=[]
object_2 = list(dict_json_weighted.keys())[6]
#for i in dict_json_weighted[object_2]['goal']:
#    if i!='message':
#        if dict_json_weighted[object_2]['goal'][i] !={}:
#            print("****** "+i+" ******")
#            for j in (dict_json_weighted[object_2]['goal'][i]):
#                if dict_json_weighted[object_2]['goal'][i][j] not in ['fail_info', {}, False]:
#                    print("°"+j)
#                    print(dict_json_weighted[object_2]['goal'][i][j])
                    #x.append({j ,(dict_json_weighted[object_2]['goal'][i][j])})
            
for i in range(len(dict_json_weighted[object_2]['log'])):
    #print(dict_json_weighted[object_2]['log'][i])
    #test_text=test_text+"\n"+dict_json_weighted[object_2]['log'][i]['text']
    #test_annot=test_annot+"\n"+str(dict_json_weighted[object_2]['log'][i]['metadata'])
    if (dict_json_weighted[object_2]['log'][i]['metadata'])!={} :
        test_data.append({dict_json_weighted[object_2]['log'][i]['text'], str(dict_json_weighted[object_2]['log'][i]['metadata'])})
        print(dict_json_weighted[object_2]['log'][i]['metadata'])
#print(test_data)
test_result = evaluate(new_model, test_data)

{'taxi': {'semi': {'arriveBy': '', 'destination': '', 'leaveAt': '', 'departure': ''}, 'book': {'booked': []}}, 'attraction': {'semi': {'name': '', 'area': '', 'type': ''}, 'book': {'booked': []}}, 'hospital': {'semi': {'department': ''}, 'book': {'booked': []}}, 'police': {'semi': {}, 'book': {'booked': []}}, 'restaurant': {'semi': {'name': 'not mentioned', 'food': 'mexican', 'area': 'not mentioned', 'pricerange': 'expensive'}, 'book': {'time': '', 'day': '', 'people': '', 'booked': []}}, 'bus': {'semi': {'arriveBy': '', 'destination': '', 'leaveAt': '', 'departure': '', 'day': ''}, 'book': {'people': '', 'booked': []}}, 'hotel': {'semi': {'name': '', 'area': '', 'pricerange': '', 'type': '', 'stars': '', 'parking': '', 'internet': ''}, 'book': {'day': '', 'stay': '', 'people': '', 'booked': []}}, 'train': {'semi': {'arriveBy': '', 'destination': '', 'leaveAt': '', 'departure': '', 'day': ''}, 'book': {'people': '', 'booked': []}}}
{'taxi': {'semi': {'arriveBy': '', 'destination': '',